In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt
from statsmodels.formula.api import ols
from scipy.stats import binom, t, sem
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt
from statsmodels.formula.api import ols
from scipy.stats import binom, t, sem
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import LeaveOneOut

data = pd.read_csv("sheets.txt", sep = "\s+")
data.head()

,day,machine,log_permeability
0,1,1,1.459
1,1,1,1.410
2,1,1,1.723
3,1,2,1.431
4,1,2,1.572


In [6]:
# Load data
sheet_data = pd.read_csv("sheets.txt", sep="\t")

# Convert day & machine to factors:
sheet_data['day'] = sheet_data['day'].astype('category')
sheet_data['machine'] = sheet_data['machine'].astype('category')

# Test for interactions
formula = 'log_permeability ~ day * machine'
L = sm.OLS.from_formula(formula, data=sheet_data).fit()

# Display interaction significance
print(sm.stats.anova_lm(L, typ=2))


               sum_sq    df         F    PR(>F)
day          0.207578   8.0  1.986502  0.065756
machine      0.000579   2.0  0.022171  0.978082
day:machine  0.243994  16.0  1.167500  0.322771
Residual     0.705335  54.0       NaN       NaN


Da p-værdien er over 0.05 for alle 3 variabler kan vi konkludere at de ikke har nogle effekt på log-permabilitet

### Exercise 2-5

In [19]:
# Predict on new sheet (from same machines on same days)
# Loop through all data (leave-one-out CV)
def MSE(formula):
    loo = LeaveOneOut()
    pred_errors = np.zeros(len(sheet_data))
    for train_index, test_index in loo.split(sheet_data):
        train_data = sheet_data.iloc[train_index]
        test_data = sheet_data.iloc[test_index]
    
    # Use the additive model
        L = sm.OLS.from_formula(formula, data=train_data).fit()
        prediction = L.predict(test_data)
        pred_errors[test_index] = test_data['log_permeability'].values - prediction
    mse1 = np.mean(pred_errors**2)
    return f"MSE1: {mse1}"
print(MSE("log_permeability ~ machine + day"))
print(MSE("log_permeability ~ day"))
print(MSE("log_permeability ~ machine"))
print(MSE("log_permeability ~ 1"))

MSE1: 0.015692990204081627
MSE1: 0.014842315972222223
MSE1: 0.015402605522682446
MSE1: 0.014649432187500002


vi ville anbefale at bruge modellen med kun dagen til at forudsige log_permabilitet